In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydub
!pip install uuid

  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6479 sha256=e1fb5a5c97cd166dbbb121329a77e39d9a525a23ada3f76daab8e0f8c59ace52
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [ ]:
%cd drive/MyDrive/acoustic-sandbox/unpacked-detection/

/content/drive/MyDrive/acoustic-sandbox/unpacked-detection


In [ ]:
!mkdir split

mkdir: cannot create directory ‘split’: File exists


In [ ]:
import pandas as pd
tsv_file = 'annotations.tsv'  # Change this to your actual TSV file path
df = pd.read_csv(tsv_file, sep='\t')

In [ ]:
df['duration_s'].mean()

1.9731221392723042

In [ ]:
import pandas as pd
from pydub import AudioSegment
import uuid

# Load the TSV file
tsv_file = 'annotations.tsv'  # Change this to your actual TSV file path
df = pd.read_csv(tsv_file, sep='\t')

# Sort the dataframe by file name and start time to ensure consecutive processing
df.sort_values(by=['wav_filename', 'start_time_s'], inplace=True)

# Initialize variables to keep track of the last end time and audio for each file
last_end_times = {}
last_end_audio = {}
clip_id = 1  # Initialize clip ID

def generate_tapename():
    # Generate a UUID for use as a unique tape name
    return str(uuid.uuid4())

for index, row in df.iterrows():
    file_name = row['wav_filename']
    start_time = int(row['start_time_s'] * 1000)  # Convert to milliseconds
    duration = int(row['duration_s'] * 1000)  # Convert to milliseconds
    end_time = start_time + duration
    year = row['date'].split('-')[0]  # Extract the year from the date
    tapename = generate_tapename()  # Generate a unique tape name

    # Check if we need to load the audio file or use the previously loaded one
    if file_name not in last_end_audio:
        audio = AudioSegment.from_wav("wav/" + file_name)
    else:
        audio = last_end_audio[file_name]

    # Extract the target signal
    target_signal = audio[start_time:end_time]

    # Save the target signal to a new file following the naming convention
    signal_file_name = f"target-ORCA_{clip_id}_{year}_{tapename}_{start_time}_{end_time}.wav"
    target_signal.export("split/" + signal_file_name, format="wav")
    clip_id += 1

    # Determine if we should extract background noise
    background_start_time = last_end_times[file_name] if file_name in last_end_times else 0
    if start_time > background_start_time:
        # Calculate the duration of the background noise
        background_duration = start_time - background_start_time
        if background_duration >= 1000:  # Check if the duration is at least 1 second
            background = audio[background_start_time:start_time]
            background_file_name = f"noise-ORCA_{clip_id}_{year}_{tapename}_{background_start_time}_{start_time}.wav"
            background.export("split/" + background_file_name, format="wav")
            clip_id += 1

    # Update the last end time and last end audio for this file
    last_end_times[file_name] = end_time
    last_end_audio[file_name] = audio

    # For the final signal, and if there's enough duration left
    if index == len(df) - 1 or df.iloc[index + 1]['wav_filename'] != file_name:
        background_end = audio[end_time:]
        if len(background_end) >= 1000:  # Check if the duration is at least 1 second
            background_end_file_name = f"noise-ORCA_{clip_id}_{year}_{tapename}_{end_time}_{len(audio)}.wav"
            background_end.export("split/" + background_end_file_name, format="wav")
            clip_id += 1

In [ ]:
!pwd

/content/drive/MyDrive/acoustic-sandbox/unpacked-detection


In [ ]:
# Split the .wav files into target and noise, removing files with duration 0
import pandas as pd
from pydub import AudioSegment
import uuid

# Load the TSV file
tsv_file = 'annotations.tsv'  # Adjust to your actual TSV file path
df = pd.read_csv(tsv_file, sep='\t')

# Sort the dataframe by file name and start time to ensure consecutive processing
df.sort_values(by=['wav_filename', 'start_time_s'], inplace=True)

# Initialize variables to keep track of the last end time and audio for each file
last_end_times = {}
last_end_audio = {}
clip_id = 1  # Initialize clip ID

def generate_tapename():
    # Generate a UUID for use as a unique tape name
    return str(uuid.uuid4())

for index, row in df.iterrows():
    file_name = row['wav_filename']
    start_time_ms = int(row['start_time_s'] * 1000)  # Convert to milliseconds
    duration_ms = int(row['duration_s'] * 1000)  # Convert to milliseconds
    end_time_ms = start_time_ms + duration_ms
    year = row['date'].split('-')[0]  # Ensure only the year is used
    tapename = generate_tapename()

    if file_name not in last_end_audio:
        audio = AudioSegment.from_wav("wav/" + file_name)
    else:
        audio = last_end_audio[file_name]

    target_signal = audio[start_time_ms:end_time_ms]

    # Check duration of the target signal and background noise before saving
    if target_signal.duration_seconds > 0:
        signal_file_name = f"target-ORCA_{clip_id}_{year}_{tapename}_{start_time_ms}_{end_time_ms}.wav"
        target_signal.export("split/" + signal_file_name, format="wav")
        clip_id += 1

    background_start_time_ms = last_end_times[file_name] if file_name in last_end_times else 0
    if start_time_ms > background_start_time_ms:
        background_duration_ms = start_time_ms - background_start_time_ms
        if background_duration_ms >= 1000:  # Check if at least 1 second
            background = audio[background_start_time_ms:start_time_ms]
            if background.duration_seconds > 0:
                background_file_name = f"noise-ORCA_{clip_id}_{year}_{tapename}_{background_start_time_ms}_{start_time_ms}.wav"
                background.export("split/" + background_file_name, format="wav")
                clip_id += 1

    last_end_times[file_name] = end_time_ms
    last_end_audio[file_name] = audio

    if index == len(df) - 1 or df.iloc[index + 1]['wav_filename'] != file_name:
        background_end = audio[end_time_ms:]
        if background_end.duration_seconds > 0 and len(background_end) >= 1000:
            background_end_file_name = f"noise-ORCA_{clip_id}_{year}_{tapename}_{end_time_ms}_{len(audio)}.wav"
            background_end.export("split/" + background_end_file_name, format="wav")
            clip_id += 1

In [ ]:
# Create separate lists of the target and noise files
import os

def list_files_by_prefix(directory, prefixes=("target", "noise")):
    # Initialize dictionaries to hold files with the given prefixes
    files_dict = {prefix: [] for prefix in prefixes}

    # Loop through files in the directory
    for file in os.listdir(directory):
        for prefix in prefixes:
            if file.startswith(prefix):
                files_dict[prefix].append(file)
                break  # Move to the next file once matched

    # Return the lists of files and their lengths
    return {prefix: (len(files_dict[prefix]), files_dict[prefix]) for prefix in prefixes}


directory_path = '/content/drive/MyDrive/acoustic-sandbox/unpacked-detection/split'
file_lists = list_files_by_prefix(directory_path)
file_lists

{'target': (4900,
  ['target-ORCA_1_2019_9127246c-3919-4093-bf83-6891b78b52ac_37345_39088.wav',
   'target-ORCA_4_2019_aa7c72bf-fca7-4311-8722-b90507eab95a_41046_42704.wav',
   'target-ORCA_7_2019_9618ab7d-8c81-4aaf-8711-390a675729b4_42917_45511.wav',
   'target-ORCA_9_2019_5900d819-b268-4256-8c83-81aef04be38f_45980_48021.wav',
   'target-ORCA_11_2019_5cb5013a-c385-4a83-aca8-aa9fbd0bdfa8_49765_52215.wav',
   'target-ORCA_14_2019_8508bbb4-8504-4f5b-a1a2-35dc06c2e727_52700_54869.wav',
   'target-ORCA_16_2019_3a160cc5-dbc1-4b91-a8d2-cd030a6046c5_55295_56400.wav',
   'target-ORCA_18_2019_a97fb235-1136-407a-9f36-fdb70f6bb1c2_0_1121.wav',
   'target-ORCA_20_2019_431a1921-28d4-4773-be69-e5a607745a07_0_1121.wav',
   'target-ORCA_22_2019_cb3618da-4ee8-4fb1-9ac7-cbac52d19a22_1694_3388.wav',
   'target-ORCA_24_2019_a1f788ee-bfad-4580-b84f-a8216686fd6f_1694_3388.wav',
   'target-ORCA_26_2019_a70604a5-59ac-4d70-900b-21765717a26e_3746_6275.wav',
   'target-ORCA_28_2019_3b859b3e-878d-4795-9e22-a6ebe6

In [ ]:
# Select script ver. 1
# Select one tenth of the data in a reproducible fashion
import random

def randomly_select_files(file_lists, seed=42, fraction=0.1):
    # Set the random seed for reproducibility
    random.seed(seed)

    # Initialize new lists for the randomly selected files
    selected_files = {key: [] for key in file_lists.keys()}

    # Loop through the original lists and select files randomly
    for key, (count, files) in file_lists.items():
        # Calculate number of files to select based on the fraction
        num_to_select = max(1, int(len(files) * fraction))  # Ensure at least one file is selected if list is not empty
        selected_files[key] = random.sample(files, num_to_select) if files else []

    # Return the new lists and their lengths
    return {prefix: (len(selected_files[prefix]), selected_files[prefix]) for prefix in selected_files}

# Assuming we're using the same file lists as before, which were empty in this case
# This is an example; in practice, you would run this after populating the lists
selected_file_lists = randomly_select_files(file_lists, seed=42)
selected_file_lists

NameError: name 'file_lists' is not defined

In [ ]:
# Move script ver. 1
# Move the files
import shutil

def move_selected_files(selected_files, new_directory):
    # Ensure the new directory exists, create it if it doesn't
    os.makedirs(new_directory, exist_ok=True)

    # Initialize a dictionary to track moved files
    moved_files = {key: [] for key in selected_files.keys()}

    # Loop through each list and move the files
    for key, (count, files) in selected_files.items():
        for file in files:
            # Construct the source and destination paths
            source_path = os.path.join('/content/drive/MyDrive/acoustic-sandbox/unpacked-detection/split', file)
            destination_path = os.path.join(new_directory, file)

            # Move the file
            shutil.move(source_path, destination_path)

            # Record the moved file
            moved_files[key].append(file)

    # Return the lists of moved files and their counts
    return {prefix: (len(moved_files[prefix]), moved_files[prefix]) for prefix in moved_files}

# Define the new directory path (this should be modified as needed)
new_directory_path = '/content/drive/MyDrive/acoustic-sandbox/unpacked-detection/split/predict-manyfiles'  # Example path, modify as needed

# Moving the files
moved_files = move_selected_files(selected_file_lists, new_directory_path)
moved_files

In [ ]:
# Select Script ver. 2
import random

def randomly_select_files(file_lists, seed=42, total_fraction=0.25, target_fraction=0.33):
    # Set the random seed for reproducibility
    random.seed(seed)

    # Calculate total number of files and then how many to select based on the desired fractions
    total_files = sum(len(files) for _, files in file_lists.values())
    total_to_select = max(1, int(total_files * total_fraction))  # Ensure at least one file is selected

    # Calculate number of target and noise files to select
    target_to_select = max(1, int(total_to_select * target_fraction))
    noise_to_select = total_to_select - target_to_select  # Remaining files are noise

    # Initialize new lists for the randomly selected files
    selected_files = {key: [] for key in file_lists.keys()}

    # Select the appropriate number of target and noise files
    if 'target' in file_lists:
        selected_files['target'] = random.sample(file_lists['target'][1], target_to_select) if file_lists['target'][1] else []
    if 'noise' in file_lists:
        selected_files['noise'] = random.sample(file_lists['noise'][1], noise_to_select) if file_lists['noise'][1] else []

    # Return the new lists and their lengths
    return {prefix: (len(selected_files[prefix]), selected_files[prefix]) for prefix in selected_files}

# Assuming we're using the file lists from the first script
selected_file_lists = randomly_select_files(file_lists, seed=42)
selected_file_lists

{'target': (786,
  ['target-ORCA_2321_1964_62320f77-7bf5-4e08-ba8d-455609b42292_46770_49460.wav',
   'target-ORCA_515_2019_f9efbabe-7127-418d-97e2-567071f33c1b_58357_59973.wav',
   'target-ORCA_4535_1997_852d7dbd-cee2-4428-b2b9-65319b353ae9_0_1928.wav',
   'target-ORCA_4285_1993_047efdcb-50c0-494a-afd5-ac69d8c8b247_0_1224.wav',
   'target-ORCA_4106_1992_29415252-2aea-4702-9ec3-9888f68ff96e_0_9543.wav',
   'target-ORCA_2886_19660_2b15959a-8bc6-45fb-96e5-3f504e91df0a_18896_20576.wav',
   'target-ORCA_2132_19640_2d35a0ae-e1f8-40e2-b1f8-d52f18f03c03_38001_39587.wav',
   'target-ORCA_7900_2020_0f5f554d-52e0-4164-be6c-ed9010d61fba_890_2402.wav',
   'target-ORCA_1789_1960_48e81a76-bf99-4114-90f5-0f8a13a225ee_1101014_1107239.wav',
   'target-ORCA_9337_2020_dc294f4e-a3bc-4294-ada0-39efcee2a14c_47850_50005.wav',
   'target-ORCA_5754_1998_6e63eb56-5006-471e-a6ed-99cd14e9c016_0_1256.wav',
   'target-ORCA_662_2019_24df3c37-7bc4-4102-b502-ae7d59ad3dd5_31358_39588.wav',
   'target-ORCA_622_2019_e9f76

In [ ]:
# Move script ver. 2
import os
import shutil

def move_selected_files(selected_files, new_directory):
    # Ensure the new directory exists, create it if it doesn't
    os.makedirs(new_directory, exist_ok=True)

    # Initialize a dictionary to track moved files
    moved_files = {key: [] for key in selected_files.keys()}

    # Loop through each list and move the files
    for key, (count, files) in selected_files.items():
        for file in files:
            # Construct the source and destination paths
            source_path = os.path.join('/content/drive/MyDrive/acoustic-sandbox/unpacked-detection/split', file)
            destination_path = os.path.join(new_directory, file)

            # Move the file
            shutil.move(source_path, destination_path)

            # Record the moved file
            moved_files[key].append(file)

    # Return the lists of moved files and their counts
    return {prefix: (len(moved_files[prefix]), moved_files[prefix]) for prefix in moved_files}

# Define the new directory path (this should be modified as needed)
new_directory_path = '/content/drive/MyDrive/acoustic-sandbox/unpacked-detection/split/bignoise'

# Moving the files
moved_files = move_selected_files(selected_file_lists, new_directory_path)
moved_files

{'target': (786,
  ['target-ORCA_2321_1964_62320f77-7bf5-4e08-ba8d-455609b42292_46770_49460.wav',
   'target-ORCA_515_2019_f9efbabe-7127-418d-97e2-567071f33c1b_58357_59973.wav',
   'target-ORCA_4535_1997_852d7dbd-cee2-4428-b2b9-65319b353ae9_0_1928.wav',
   'target-ORCA_4285_1993_047efdcb-50c0-494a-afd5-ac69d8c8b247_0_1224.wav',
   'target-ORCA_4106_1992_29415252-2aea-4702-9ec3-9888f68ff96e_0_9543.wav',
   'target-ORCA_2886_19660_2b15959a-8bc6-45fb-96e5-3f504e91df0a_18896_20576.wav',
   'target-ORCA_2132_19640_2d35a0ae-e1f8-40e2-b1f8-d52f18f03c03_38001_39587.wav',
   'target-ORCA_7900_2020_0f5f554d-52e0-4164-be6c-ed9010d61fba_890_2402.wav',
   'target-ORCA_1789_1960_48e81a76-bf99-4114-90f5-0f8a13a225ee_1101014_1107239.wav',
   'target-ORCA_9337_2020_dc294f4e-a3bc-4294-ada0-39efcee2a14c_47850_50005.wav',
   'target-ORCA_5754_1998_6e63eb56-5006-471e-a6ed-99cd14e9c016_0_1256.wav',
   'target-ORCA_662_2019_24df3c37-7bc4-4102-b502-ae7d59ad3dd5_31358_39588.wav',
   'target-ORCA_622_2019_e9f76